# Tarefa 9
Implementar um algoritmo genético para resolver um problema de minimização.

## Autores
Renan Mateus Bernardo Nascimento
<br/>Vinícius Magalhães D'Assunção

In [1]:
import numpy as np
import random

## Classe Cromossomo
Define o TAD de cromossomo, que possui como atributos uma lista de alelos e um valor de aptidão.

In [2]:
class Cromossomo:
    
    def __init__(self, qtd_alelos):
        self.alelos = np.zeros(qtd_alelos)
        self.aptidao = 0.0

## Classe Ag
Possui a implementação do algoritmo genético, possuindo os métodos para de população, cálculo de aptidão, seleção, cruzamento e mutação.

In [49]:
class Ag:

    def __init__(self, tam_populacao, inicio, fim, alfa_cruzamento, tax_cruzamento, tax_mutacao, max_geracoes):
        self.tam_populacao = tam_populacao
        # valor inicial para alelos do cromossomo
        self.inicio = inicio
        # Valor final para alelos do cromossomo
        self.fim = fim
        # Valor porcentagem do cromossomo que será utilizada dos pais no cruzamento
        self.alfa_cruzamento = alfa_cruzamento
        self.tax_cruzamento = tax_cruzamento
        self.tax_mutacao = tax_mutacao
        self.max_geracoes = max_geracoes
        # offset da função de aptidão
        self.OFFSET = 1500
        self.QTD_ALELOS = 2
        self.populacao = []

    '''
    ' Inicializa a lista de população
    '''
    def popular(self):
        for c in range(0, self.tam_populacao):
            cromo = Cromossomo(self.QTD_ALELOS)
            for a in range(0, len(cromo.alelos)):
                cromo.alelos[a] = random.randrange(self.inicio, self.fim + 1)
            cromo.aptidao = self.calcAptidao(cromo.alelos[0], cromo.alelos[1])
            self.populacao.append(cromo)

    '''
    ' Calcula a função de aptidão de toda a população
    '''
    def calcAptidao(self, x1, x2):
        return self.OFFSET + (x2 + 47) * np.sin(np.sqrt(abs(x2 + x1/2 +47))) + x1 * np.sin(np.sqrt(abs(x1 - (x2 + 47))))
        
    '''
    ' Faz a seleção da população utilizando o método da roleta
    ' @return selecao
    '''
    def selecionar(self):
        selecao = []
        roleta = {}
        total = 0
        
        # Adiciona índice do cromossomo à roleta
        for i, c in enumerate(self.populacao):
            # A roleta possui a tupla inicio_intervalo e fim_intervalo como índice do dicionário
            # O valor corresponde ao índice do vetor de população
            roleta[(total, total + c.aptidao)] = i
            total += c.aptidao
        
        # Sorteia tam_populacao/2 números para selecionar na roleta
        for i in range(0, self.tam_populacao):
            # Sorteia o número entre 0(fechado) e a aptidão total(aberto)
            num = random.uniform(0, total)
            for intervalo, c in roleta.items():
                if num >= intervalo[0] and num < intervalo[1]:
                    selecao.append(c)
        return selecao
    
    '''
    ' Faz o cruzamento utilizando o método aritmético
    '''
    def cruzar(self, selecao):
        selecao = self.selecionar()
        filhos = []
        for i in range(0, int(len(selecao)/2)):
            (pai, pai2) = (self.populacao[selecao[2 * i]], self.populacao[selecao[2 * i + 1]])
            # Sorteia um número de 0 a 1 com duas casas decimais
            num = random.randrange(0, 100) / 100
            if num <= self.tax_cruzamento:
                # Se for menor que a taxa de cruzamento, faz o cruzamento aritmético
                qtd_alelos = len(pai.alelos)
                # Cria os cromossomos
                f1 = Cromossomo(qtd_alelos)
                f2 = Cromossomo(qtd_alelos)
                f1.alelos = self.alfa_cruzamento * pai.alelos + (1 - self.alfa_cruzamento) * pai2.alelos
                f2.alelos = (1 - self.alfa_cruzamento) * pai.alelos + self.alfa_cruzamento * pai2.alelos
                f1.aptidao = self.calcAptidao(f1.alelos[0], f1.alelos[1])
                f2.aptidao = self.calcAptidao(f2.alelos[0], f2.alelos[1])
                # Adiciona a lista de filhos
                filhos.append(f1)
                filhos.append(f2)
            else:
                # Senão apenas propaga os pais
                filhos.append(pai)
                filhos.append(pai2)
        return filhos
    
    '''
    ' Faz a mutação utilizando o método CREEP
    '''
    def mutar(self, filhos):
        n_casas = 1000 # Número de casas decimais para avaliar a mutação
        for cromo in filhos:
            for i in range(0, len(cromo.alelos)):
                # Sorteia um número de 0 a 1 com 3 casas decimais
                num = random.randrange(0, n_casas) / n_casas
                if num <= self.tax_mutacao:
                    delta = random.randrange(self.inicio, self.fim)
                    cromo.alelos[i] += delta

    '''
    ' Executa max_geracoes vezes o algoritmo genético
    '''
    def executar(self):
        self.popular()
        self.imprimirPopulacao()
        selecao = self.selecionar()
        filhos = self.cruzar(selecao)
        self.mutar(filhos)
        self.populacao = filhos
        print("")
        self.imprimirPopulacao()

    def imprimirPopulacao(self):
        for c in self.populacao:
            print(str(c.alelos[0]) + '\t' + str(c.alelos[1]) + '\t' + str(c.aptidao))

In [48]:
tam_populacao, inicio, fim, alfa, tax_cruzamento, tax_mutacao, max_geracoes = (12, -600, 600, 0.6, 0.5, 0.008, 10)
ag = Ag(tam_populacao, inicio, fim, alfa, tax_cruzamento, tax_mutacao, max_geracoes)

ag.executar()

-74.0	505.0	1151.6788901
563.0	143.0	1803.76087659
-511.0	22.0	2004.32072848
-156.0	557.0	909.736345722
256.0	495.0	1631.15721366
446.0	-310.0	1933.81893187
-327.0	-174.0	1296.52225174
568.0	589.0	1462.61950419
-523.0	456.0	1282.07384687
-483.0	338.0	1734.64361366
-262.0	-191.0	1870.23440809
-591.0	549.0	843.203182912

70.0	121.0	1634.97671698
236.0	277.0	1777.14515498
-494.2	211.6	1069.07313424
-499.8	148.4	1202.36874048
256.0	495.0	1631.15721366
-262.0	-191.0	1870.23440809
70.0	121.0	1634.97671698
236.0	277.0	1777.14515498
-494.2	211.6	1069.07313424
-499.8	148.4	1202.36874048
568.0	589.0	1462.61950419
256.0	495.0	1631.15721366
